In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

## Imports and Library Setup

In [ ]:
import {
  createToken,
  Lexer,
  CstParser,
  CstNode,
  IToken,
  ILexingResult,
  TokenType,
} from "chevrotain";
import readlineSync from "readline-sync";

# A Simple Calculator

This file shows the implementation of a *symbolic calculator* using `Ply`.  The grammar for the language implemented by this parser is as follows:
$$
\begin{array}{lcl}
  \texttt{stmnt}   & \rightarrow & \;\texttt{IDENTIFIER}\; \texttt{':='} \; \;\texttt{expr}\; \texttt{';'} \\
                   & \mid        & \;\texttt{expr}\; \texttt{';'} \\[0.2cm]   
  \texttt{expr}    & \rightarrow & \;\texttt{expr}\; \texttt{'+'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{expr}\; \texttt{'-'} \; \texttt{product}  \\
                   & \mid        & \;\texttt{product}                                  \\[0.2cm]
  \texttt{product} & \rightarrow & \;\texttt{product}\; \texttt{'*'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{product}\; \texttt{'/'} \;\texttt{factor} \\
                   & \mid        & \;\texttt{factor}                                   \\[0.2cm]
  \texttt{factor}  & \rightarrow &   \texttt{'('} \; \texttt{expr} \;\texttt{')'}      \\
                   & \mid        & \;\texttt{NUMBER}                                   \\
                   & \mid        & \;\texttt{IDENTIFIER}                               
\end{array}
$$

## Specification of the Scanner

We define the tokens of the calculator language using Chevrotain’s `createToken`.  
Whitespace is skipped by the lexer and does not appear in the token stream.

In [ ]:
// Lexer Definition (entspricht t_NUMBER, t_IDENTIFIER, etc.)

interface CalcTokens {
  NumberTok: TokenType;
  Identifier: TokenType;
  Assign: TokenType;
  Plus: TokenType;
  Minus: TokenType;
  Mul: TokenType;
  Div: TokenType;
  LParen: TokenType;
  RParen: TokenType;
  Semicolon: TokenType;
  WhiteSpace: TokenType;
}

function createCalcLexer(): {
  allTokens: TokenType[];
  tokens: CalcTokens;
  lexer: Lexer;
} {
  const NumberTok: TokenType = createToken({
    name: "NUMBER",
    pattern: /(0|[1-9][0-9]*)(\.[0-9]*)?/,
  });
  const Identifier: TokenType = createToken({
    name: "IDENTIFIER",
    pattern: /[a-zA-Z][a-zA-Z0-9_]*/,
  });
  const Assign: TokenType = createToken({ name: "ASSIGN", pattern: /:=/ });
  const Plus: TokenType = createToken({ name: "PLUS", pattern: /\+/ });
  const Minus: TokenType = createToken({ name: "MINUS", pattern: /-/ });
  const Mul: TokenType = createToken({ name: "MUL", pattern: /\*/ });
  const Div: TokenType = createToken({ name: "DIV", pattern: /\// });
  const LParen: TokenType = createToken({ name: "LPAREN", pattern: /\(/ });
  const RParen: TokenType = createToken({ name: "RPAREN", pattern: /\)/ });
  const Semicolon: TokenType = createToken({
    name: "SEMICOLON",
    pattern: /;/,
  });
  const WhiteSpace: TokenType = createToken({
    name: "WhiteSpace",
    pattern: /[ \t\r]+/,
    group: Lexer.SKIPPED,
  });

  const allTokens: TokenType[] = [
    WhiteSpace,
    Assign,
    Plus,
    Minus,
    Mul,
    Div,
    LParen,
    RParen,
    Semicolon,
    NumberTok,
    Identifier,
  ];

  return {
    allTokens,
    tokens: {
      NumberTok,
      Identifier,
      Assign,
      Plus,
      Minus,
      Mul,
      Div,
      LParen,
      RParen,
      Semicolon,
      WhiteSpace,
    },
    lexer: new Lexer(allTokens, { positionTracking: "onlyOffset" }),
  };
}

const {
  allTokens,
  tokens,
  lexer: CalcLexer,
}: {
  allTokens: TokenType[];
  tokens: CalcTokens;
  lexer: Lexer;
} = createCalcLexer();

For debugging purposes, we provide a helper function `tokenizeCalc` that returns the list of token images (strings).

In [ ]:
function tokenizeCalc(input: string): string[] {
  const lexingResult: ILexingResult = CalcLexer.tokenize(input);

  if (lexingResult.errors.length > 0) {
    throw new Error(`Lexing errors: ${lexingResult.errors[0].message}`);
  }

  return lexingResult.tokens.map((t: IToken) => t.image);
}

In [ ]:
// Example:
console.log(tokenizeCalc("x := 3 + 4 * 2;"));

## Specification of the Parser

We now implement the parser using `CstParser`.  
The left‑recursive grammar rules (`expr → expr '+' product` etc.) are rewritten in an LL(1)‑friendly way using `MANY` to represent repetition.

The resulting rules are:

- `stmnt → IDENTIFIER ASSIGN expr SEMICOLON | expr SEMICOLON`
- `expr → product (('+' | '-') product)*`
- `product → factor (('*' | '/') factor)*`
- `factor → '(' expr ')' | NUMBER | IDENTIFIER`

In [ ]:
class CalcParser extends CstParser {
  constructor() {
    super(allTokens);
    this.performSelfAnalysis();
  }

  // stmnt -> IDENTIFIER ASSIGN expr SEMICOLON
  //        | expr SEMICOLON
  public stmnt = this.RULE("stmnt", () => {
    this.OR([
      {
        ALT: () => {
          this.CONSUME(tokens.Identifier);
          this.CONSUME(tokens.Assign);
          this.SUBRULE(this.expr);
          this.CONSUME(tokens.Semicolon);
        },
      },
      {
        ALT: () => {
          this.SUBRULE1(this.expr);
          this.CONSUME1(tokens.Semicolon);
        },
      },
    ]);
  });

  // expr -> product (('+' | '-') product)*
  public expr = this.RULE("expr", () => {
    this.SUBRULE(this.product);
    this.MANY(() => {
      this.OR([
        { ALT: () => this.CONSUME(tokens.Plus) },
        { ALT: () => this.CONSUME(tokens.Minus) },
      ]);
      this.SUBRULE2(this.product);
    });
  });

  // product -> factor (('*' | '/') factor)*
  public product = this.RULE("product", () => {
    this.SUBRULE(this.factor);
    this.MANY(() => {
      this.OR([
        { ALT: () => this.CONSUME(tokens.Mul) },
        { ALT: () => this.CONSUME(tokens.Div) },
      ]);
      this.SUBRULE2(this.factor);
    });
  });

  // factor -> '(' expr ')' | NUMBER | IDENTIFIER
  public factor = this.RULE("factor", () => {
    this.OR([
      {
        ALT: () => {
          this.CONSUME(tokens.LParen);
          this.SUBRULE(this.expr);
          this.CONSUME(tokens.RParen);
        },
      },
      { ALT: () => this.CONSUME(tokens.NumberTok) },
      { ALT: () => this.CONSUME(tokens.Identifier) },
    ]);
  });
}

const calcParser: CalcParser = new CalcParser();
const CalcBaseVisitor = calcParser.getBaseCstVisitorConstructor();

## Abstract Syntax Tree (AST)

We represent expressions and statements using simple TypeScript types:

- `Expr` for arithmetic expressions.
- `Stmnt` for either assignments or plain expression statements.
- `Env` as the variable environment mapping identifiers to numeric values.


In [ ]:
type Expr =
  | { kind: "num"; value: number }
  | { kind: "var"; name: string }
  | { kind: "binop"; op: "+" | "-" | "*" | "/"; left: Expr; right: Expr };

type Stmnt =
  | { kind: "assign"; name: string; expr: Expr }
  | { kind: "expr"; expr: Expr };

type Env = Record<string, number>;

## Visitor: From CST to AST

The `CalcAstVisitor` traverses the Concrete Syntax Tree (CST) produced by the parser and builds the typed AST defined above.


In [ ]:
class CalcAstVisitor extends CalcBaseVisitor {
  constructor() {
    super();
    this.validateVisitor();
  }

  public stmnt(ctx: {
    IDENTIFIER?: IToken[];
    expr?: CstNode[];
  }): Stmnt {
    if (ctx.IDENTIFIER && ctx.IDENTIFIER.length > 0) {
      const name: string = ctx.IDENTIFIER[0].image;
      const exprNode: Expr = this.visit(ctx.expr![0]) as Expr;
      return { kind: "assign", name, expr: exprNode };
    }
    const exprNode: Expr = this.visit(ctx.expr![0]) as Expr;
    return { kind: "expr", expr: exprNode };
  }

  public expr(ctx: {
    product: CstNode[];
    PLUS?: IToken[];
    MINUS?: IToken[];
  }): Expr {
    let result: Expr = this.visit(ctx.product[0]) as Expr;

    if (ctx.product.length > 1) {
      for (let i = 1; i < ctx.product.length; i++) {
        const right: Expr = this.visit(ctx.product[i]) as Expr;
        const plusTok: IToken | undefined =
          ctx.PLUS && ctx.PLUS[i - 1] ? ctx.PLUS[i - 1] : undefined;
        const minusTok: IToken | undefined =
          ctx.MINUS && ctx.MINUS[i - 1] ? ctx.MINUS[i - 1] : undefined;

        if (plusTok) {
          result = { kind: "binop", op: "+", left: result, right };
        } else if (minusTok) {
          result = { kind: "binop", op: "-", left: result, right };
        }
      }
    }

    return result;
  }

  public product(ctx: {
    factor: CstNode[];
    MUL?: IToken[];
    DIV?: IToken[];
  }): Expr {
    let result: Expr = this.visit(ctx.factor[0]) as Expr;

    if (ctx.factor.length > 1) {
      for (let i = 1; i < ctx.factor.length; i++) {
        const right: Expr = this.visit(ctx.factor[i]) as Expr;

        const mulTok: IToken | undefined =
          ctx.MUL && ctx.MUL[i - 1] ? ctx.MUL[i - 1] : undefined;
        const divTok: IToken | undefined =
          ctx.DIV && ctx.DIV[i - 1] ? ctx.DIV[i - 1] : undefined;

        if (mulTok) {
          result = { kind: "binop", op: "*", left: result, right };
        } else if (divTok) {
          result = { kind: "binop", op: "/", left: result, right };
        }
      }
    }

    return result;
  }

  public factor(ctx: {
    expr?: CstNode[];
    NUMBER?: IToken[];
    IDENTIFIER?: IToken[];
  }): Expr {
    if (ctx.expr && ctx.expr.length > 0) {
      return this.visit(ctx.expr[0]) as Expr;
    }
    if (ctx.NUMBER && ctx.NUMBER.length > 0) {
      const value: number = parseFloat(ctx.NUMBER[0].image);
      return { kind: "num", value };
    }
    const name: string = ctx.IDENTIFIER![0].image;
    return { kind: "var", name };
  }
}

## Evaluation of Expressions and Statements

We now implement the evaluation of expressions and statements with respect to an environment `Env`.

In [ ]:
function evalExpr(e: Expr, env: Env): number {
  switch (e.kind) {
    case "num":
      return e.value;
    case "var": {
      const v: number | undefined = env[e.name];
      if (v === undefined) {
        throw new Error(`Undefined variable: ${e.name}`);
      }
      return v;
    }
    case "binop": {
      const left: number = evalExpr(e.left, env);
      const right: number = evalExpr(e.right, env);
      switch (e.op) {
        case "+":
          return left + right;
        case "-":
          return left - right;
        case "*":
          return left * right;
        case "/":
          return left / right;
      }
    }
  }
}

In [ ]:
function execStmnt(s: Stmnt, env: Env): number | undefined {
  if (s.kind === "assign") {
    const value: number = evalExpr(s.expr, env);
    env[s.name] = value;
    return value;
  }
  // Ausdrucksstatement: Wert berechnen und zurückgeben
  return evalExpr(s.expr, env);
}

## The `parseCalc` Function

This function ties together lexing, parsing, and CST→AST transformation for a single statement.

In [ ]:
function parseCalc(input: string): Stmnt {
  const lexingResult: ILexingResult = CalcLexer.tokenize(input);

  if (lexingResult.errors.length > 0) {
    throw new Error(`Lexing errors: ${lexingResult.errors[0].message}`);
  }

  calcParser.input = lexingResult.tokens;
  const cst: CstNode = calcParser.stmnt();

  if (calcParser.errors.length > 0) {
    throw new Error(`Parsing errors: ${calcParser.errors[0].message}`);
  }

  const visitor: CalcAstVisitor = new CalcAstVisitor();
  return visitor.visit(cst) as Stmnt;
}

## Input Variable and `calc` Function

Instead of reading from standard input in a loop, we use a single input string defined in a variable `inputProgram` in the cell before calling `calc()`.


Example input program for the calculator.
You can change this string and re-run calc().

In [ ]:
// Example input program for the calculator.
// You can change this string and re-run calc().
const inputProgram: string = `
x := 3 + 4 * 2;
y := x * 40;
y + 1;
y - 31;
y := 1;
`;

In [ ]:
function calc(): void {
   const env: Env = {};

  const lines: string[] = inputProgram
    .split("\n")
    .map((l: string): string => l.trim())
    .filter((l: string): boolean => l.length > 0);

  for (const line of lines) {
    try {
      const stmnt: Stmnt = parseCalc(line);
      const value: number | undefined = execStmnt(stmnt, env);
      console.log(`Input: ${line}`);
      if (value !== undefined) {
        console.log(`Result: ${value}`);
      }
    } catch (e) {
      console.error(`Error processing '${line}':`, e);
    }
  }

  console.log("Final environment:", env);
}

Run the calculator once for the given inputProgram.

In [ ]:
calc();